# Modelling

### Import Packages

In [313]:
sys.path.insert(0, '/NHL_Game_Prediction/Functions/')
from functions.modelling_functions import *
import numpy as np
import pickle
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection

In [314]:
import warnings
warnings.filterwarnings("ignore")

### Load Data

In [315]:
nhl_df = pd.read_csv('../Data/preprocessed_df.csv')
team_info = pd.read_csv('../Data/team_info.csv')

In [316]:
nhl_df = nhl_df.sort_values(by='date_time_GMT_x').reset_index(drop=True)

In [318]:
# Get relevant columns
final_df = get_modelling_columns(nhl_df)

In [319]:
y = final_df['target']
X = final_df.drop(columns=['target'])

# Scale Data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)
scaled_df = pd.DataFrame(scaled_data, columns=X.columns)

In [320]:
# Split Data
split_marker = round(0.75 * len(X))

X_train = scaled_df[:split_marker]
Y_train = y.iloc[:split_marker]
X_test = scaled_df[split_marker:]
Y_test = y.iloc[split_marker:]

#### Logistic Regression

In [ ]:
lr = LogisticRegression(class_weight='balanced', C=0.1, penalty='l1', solver='liblinear')
lr.fit(X_train, Y_train)
y_pred = lr.predict(X_test)
proba = lr.predict_proba(X_test)
print("Accuracy is", accuracy_score(Y_test,y_pred))
print("Precision is", precision_score(Y_test,y_pred))
print("Recall is", recall_score(Y_test,y_pred))
print("f1 score is", f1_score(Y_test,y_pred))

In [78]:
# Exports results to the betting phase
to_betting = process_predictions(y_pred, proba, nhl_df, team_info)
to_betting.to_csv('../Data/predictions.csv')